In [1]:
import sys
sys.path.append("..")
import imagenet_load
from importlib import reload
reload(imagenet_load)
import tarfile
import numpy as np
import imread
%pylab inline
import imread
from imread import imread_from_blob
from imagenet_load import orient
import multigpu
reload(multigpu)
import filter_gen
import conv
reload(conv)
import gc
import logging
import dill
from scipy.misc import imresize
import opt
reload(opt)
import exputil
reload(exputil)
from sklearn import metrics
import time
import scipy.linalg
from numba import jit
import concurrent.futures as fs
from scipy import stats
import softmax
reload(softmax)

Populating the interactive namespace from numpy and matplotlib


/data/vaishaal/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['imread']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


<module 'softmax' from '/data/vaishaal/pictureweb/conv/softmax.py'>

In [11]:
%load_ext line_profiler


In [6]:
def softmax_gn(X_train, y_train, X_test, y_test, XTX, w_init=None, step_size=1, multiplier=1e-2, num_classes=1000, numiter=50, verbose=True):
        ''' Implementation of gauss-newton quassi-newton optimization algorithm
            with softmax objective
        '''
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)
        gmat = (1.0/X_train.shape[0])*XTX
        lambdav = multiplier*np.trace(gmat)/gmat.shape[0]
        num_idx = np.diag_indices(gmat.shape[0])
        gmat[num_idx] += lambdav
        w = np.zeros((num_classes, gmat.shape[0]))
        if (w_init != None):
            w = w_init
        num_samples = X_train.shape[0]
        onehot = lambda x: np.eye(num_classes)[x]
        class_matrix = np.eye(num_classes)
        y_train_hot = class_matrix[y_train]
        y_test_hot = class_matrix[y_test]
        for k in range(numiter):
            print("Computing Trian Preds")
            train_preds  = w.dot(X_train.T).T # 1million x 1000
            
            print("Normalizing trian preds")
            train_preds = train_preds - np.max(train_preds, axis=1)[:,np.newaxis]
            train_preds = np.exp(train_preds)
            train_preds = train_preds/(np.sum(train_preds, axis=1)[:,np.newaxis])
            train_preds = y_train_hot - train_preds
            
            print("Computing gradient")
            grad = (1.0/num_samples)*(X_train.T.dot(train_preds).T) - lambdav*w
            
            print("Solving for step")
            w = w + step_size * (np.linalg.solve(gmat, grad.T)).T
            
            print("Computing metrics")
            y_train_pred = np.argmax(w.dot(X_train.T).T, axis=1)
            y_test_pred = np.argmax(w.dot(X_test.T).T, axis=1)
            train_acc = metrics.accuracy_score(y_train, y_train_pred)
            test_acc = metrics.accuracy_score(y_test, y_test_pred)
            if (verbose):
              print("Iter: {0}, Train Accuracy: {1}, Test Accuracy: {2}".format(k, train_acc, test_acc))
        return y_train_pred, y_test_pred, w


In [5]:
alexnet_train = np.load("/data/vaishaal/pictureweb/pywren_kernels/imagenet_train_alexnet_fc7.npz")

X_train = alexnet_train["X_train"]
y_train = alexnet_train["y_train"]

X_test = np.load("/data/vaishaal/pictureweb/pywren_kernels/imagenet_test_features_alexnet_fc7.npy")
y_test  = np.load("/data/vaishaal/pictureweb/pywren_kernels/imagenet_test_labels.npy")

%time XtX = X_train.T.dot(X_train)

CPU times: user 19min 1s, sys: 1.4 s, total: 19min 3s
Wall time: 23.9 s


In [13]:
%lprun  softmax_gn(X_train[:1000,:], y_train[:1000], X_test, y_test, XtX, multiplier=1e-3, step_size=10o)

Computing Trian Preds
Normalizing trian preds
Computing gradient
Solving for step
Computing metrics
Iter: 0, Train Accuracy: 1.0, Test Accuracy: 0.11524
Computing Trian Preds
Normalizing trian preds
Computing gradient
Solving for step
Computing metrics
Iter: 1, Train Accuracy: 1.0, Test Accuracy: 0.11588
Computing Trian Preds
Normalizing trian preds
Computing gradient
Solving for step
*** KeyboardInterrupt exception caught in code being profiled.